## 데이터 수집

- https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_79c6f1a4-56dd-492e-ad67-c5acba0304d2/5# 에서 무료 제공
- csv파일 3개 다운로드

## 데이터 전처리

### 인코딩 
- 인코딩 utf-8

In [ ]:
import pandas as pd
import chardet

# 기존 파일 리스트
original_paths = [
    "TB_RECIPE_SEARCH-220701.csv",
    "TB_RECIPE_SEARCH-20231130.csv",
    "TB_RECIPE_SEARCH_241226.csv"
]

def encoding(filepath, num_bytes=10000):
    # 인코딩 감지
    with open(filepath, 'rb') as f:
        raw = f.read(num_bytes)
    result = chardet.detect(raw)
    return result['encoding']

# 인코딩 변환 및 덮어쓰기 저장
for path in original_paths:
    try:
        detected_encoding = encoding(path)
        print(f"[감지됨] {path} → 인코딩: {detected_encoding}")

        # 감지된 인코딩으로 열기
        with open(path, 'r', encoding=detected_encoding, errors='replace') as f:
            df = pd.read_csv(f)

        # UTF-8 인코딩 덮어쓰기 저장
        df.to_csv(path, index=False, encoding='utf-8-sig')

        print(f"[저장 완료] {path} → UTF-8 인코딩 완료")

    except Exception as e:
        print(f"[실패] {path}: {e}")

[감지됨] TB_RECIPE_SEARCH_241226.csv → 인코딩: utf-8
[저장 완료] TB_RECIPE_SEARCH_241226.csv → UTF-8 인코딩 완료


### 파일 통합
- 3개 파일 통합
- .json로 변환
- github push 용량 제한으로 다시 4분할

In [1]:
import pandas as pd
import numpy as np

# 파일 리스트
csv_files = [
    "TB_RECIPE_SEARCH-220701.csv",
    "TB_RECIPE_SEARCH-20231130.csv",
    "TB_RECIPE_SEARCH_241226.csv"
]

# 하나의 DataFrame으로 결합
df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file, encoding='utf-8-sig')
        df_list.append(df)
        print(f"[읽기 완료] {file}")
    except Exception as e:
        print(f"[오류] {file}: {e}")

# DataFrame 병합
merged_df = pd.concat(df_list, ignore_index=True)

# 4분할 저장
chunks = np.array_split(merged_df, 4) 
for i, chunk in enumerate(chunks, 1):
    output_file = f"recipe_mangae_{i}.json"
    chunk.to_json(output_file, orient='records', force_ascii=False, indent=2)
    print(f"[저장 완료] {output_file} → {len(chunk)}개 레시피")

[읽기 완료] TB_RECIPE_SEARCH-220701.csv
[읽기 완료] TB_RECIPE_SEARCH-20231130.csv
[읽기 완료] TB_RECIPE_SEARCH_241226.csv


c:\Users\User\miniconda3\envs\boracay\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[저장 완료] recipe_mangae_1.json → 84146개 레시피
[저장 완료] recipe_mangae_2.json → 84146개 레시피
[저장 완료] recipe_mangae_3.json → 84146개 레시피
[저장 완료] recipe_mangae_4.json → 84145개 레시피
